# Requirement
1. Write a python script to pull some required information from a web page

2. Clean the pulled informationand store it in a csv file

3. The python script should take url as input and output the csv file with pulled, cleaned required information.

Name of the restaurant

How many reviews are there in total

Reviews

  - Review_text

  - Reviewer

  - Rating



Create a csv file with the above details.

Bonus: If you can extend the above script to crawl multiple restaurant pages, pull all the above review information and write them into a csv file, you will get bonus marks. Very tough!!!

# Code

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import sys
import os
import csv

In [ ]:
# Label = aria-label
#  undefined list__09f24__ynIEd
# Comment class =  css-1q2nwpv

#  css-1q2nwpv. css-1qn0b6x
#       Rating: . css-10n911v   ... .five-stars__09f24__mBKym
#       Comment: . css-9ul5p9.comment__09f24__D0cxf css-qgunke. raw__09f24__T4Ezm   <- Content of this /span
#       Name: . css-174a15u.css-1w3ky6t.css-1u1p5a2. css-1qn0b6x   <- aria-label

In [38]:
URL = "https://www.yelp.ca/biz/pho-con-bo-north-york?page_src=related_bizes"

def get_page():
    global url
    
    print('Enter url of a medium article: ')
    # url = input()
    # if url is None:
    url = URL
    if not re.match(r'https?://www.yelp.ca/',url):
        print('Please enter a valid website, or make sure it is a yelp url')
        sys.exit(1)
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def clean(text):
    rep = {"<br>": "\n", "<br/>": "\n", "<li>":  "\n"}
    rep = dict((re.escape(k), v) for k, v in rep.items()) 
    pattern = re.compile("|".join(rep.keys()))
    text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
    text = re.sub('\<(.*?)\>', '', text)
    return text

def collect_text(soup):
    text = f'url: {url}\n\n'
    para_text = soup.find_all('p')
    print(f"paragraphs text = \n {para_text}")
    for para in para_text:
        text += f"{para.text}\n\n"
    return text

def collect_name_res(soup):

    restaurant_text = soup.find("h1")
    restaurant_name = restaurant_text.text.strip()
    # url_parts = url.split("/biz/")
    # restaurant_name_parts = url_parts[1].split("-")
    # restaurant_name = " ".join(restaurant_name_parts[:-1])

    return restaurant_name

def collect_the_total_review(soup):
    reviews_element = soup.find("p", class_="css-foyide")
    print(reviews_element)
    reviews_text = reviews_element.text.split(' ')[0]
    return reviews_text

def collect_name(soup):
    text = []
    name_text = soup.select('div[class*="user-passport-info"] span[class*="fs-block"] a')
    for name in name_text:
        val = name.text
        text.append(val)
    return text

def collect_review(soup):
    text = []
    name_text = soup.select('p[class*="comment__09f24__D0cxf"] span')
    for name in name_text:
        val = name.text
        text.append(val)
    return text

def collect_stars(soup):
    text = []
    # name_text = soup.select('span[class*=" css-1d8srnw"] div[class*="stars__09f24__mBKym"]')
    name_text = soup.select('#reviews > section > div.css-1qn0b6x > ul > li > div > div.css-10n911v > div > div > span > div')
    for name in name_text:
        val = name['aria-label']
        text.append(val)
    return text

def save_file(text):
    if not os.path.exists('./scraped_articles'):
        os.mkdir('./scraped_articles')
    name = url.split("/")[-1]
    print(name)
    fname = f'scraped_articles/{name}.txt'
    with open(fname, 'w') as file:
        file.write(text)
        
if __name__ == '__main__':
    page = get_page()
    # text = collect_text(get_page())
    res_name = collect_name_res(page)
    total_review = collect_the_total_review(page)
    names_customer = collect_name(page)
    reviews_customer = collect_review(page)
    stars_customer = collect_stars(page)
 
    # Define the data
    # Create a list of lists to represent the CSV data
    csv_data = [
        ["restaurant name", res_name],
        ["total review", total_review],
        names_customer,
        reviews_customer,
        stars_customer
    ]

    # Define the output file path
    output_csv_path = "restaurant_reviews.csv"

    # Write the CSV data to the file
    with open(output_csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(csv_data)

    print(f"CSV file saved to: {output_csv_path}")

    # Code ends here
    # save_file(text)

Enter url of a medium article: 
<p class="css-foyide">102 reviews</p>
CSV file saved to: restaurant_reviews.csv
